In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import some necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import dataset

titanic_data = pd.read_csv("/kaggle/input/titanicdataset-traincsv/train.csv")
titanic_data.head()

In [ ]:
# Shape of the dataset

titanic_data.shape

There are 891 Rows & 12 Columns, According to me columns=["PassangerId","Name","Ticket"] are not useful for the models. So, let's first drop it.

In [ ]:
# Droping Unusable Columns

titanic_data.drop(columns = ["PassengerId", "Name", "Ticket"], axis=1, inplace=True)

print("Shape of the data --> ",titanic_data.shape)
titanic_data.head()

Now our data is ready for EDA

In [ ]:
# Quick information about dataset

titanic_data.info()

It seems that there are missing values in our dataset. Let's check how many missing values are available

In [ ]:
# Missing Values

titanic_data.isnull().sum()

Columns = ["Age", "Cabin", "Embarked"] have null values. Let's check how many percentages (%) they are covering from the datset.

In [ ]:
column_data = titanic_data.isna().sum().keys().tolist()
value_data = titanic_data.isna().sum().values.tolist()

missing_data = pd.DataFrame(list(zip(column_data, value_data)),columns=["Columns", "Missing Values"],index=list(range(1,10)))
missing_data["Missing %"] = round(((missing_data["Missing Values"]/titanic_data.shape[0])*100),2)

print("\nMissing Values for Titanic Data :-")
missing_data

"Cabin" is covering 77.10 % of null values which is very high. and other are covering less null values.

In [ ]:
# Let's describe the dataset

titanic_data.describe()

Survive is our target column.

It seems that some have age and some doesn't have age.

It seems that Age and Fare column have skewness. Survived is our target column and Pclass,SibSp & Parch looks as classification. So we need to focus on Age & Fare columns for filling null values.

In [ ]:
# Let's check the "0" values if there are present

for col in titanic_data:
           print(col +"  "+str(titanic_data[titanic_data[col]==0].shape[0]))
        

I discussed about all column in which 0 is present but Fare column is not a classification column and it has 0 values. Let's analyze it.

In [ ]:
# Let's check Fare column where 0 values are present

titanic_data[titanic_data["Fare"]==0]

In [ ]:
# Let's describe the categorical data

titanic_data.describe(include='object')

The frequency of Male is higher than Female, Males have covered 65% data. In Embarked S covered the 644 out of 889.

In [ ]:
# Let's check pairplot with Survived

plt.figure(figsize=(7,7))
sns.pairplot(titanic_data,hue="Survived")

In [ ]:
# Analyzing 

sns.catplot(x="Age",y="Sex",data=titanic_data,hue="Survived")

It seems that Females are more survive than Males

In [ ]:
# Let's Handle the Missing Data


# Dropna Embarked
titanic_data.dropna(subset=["Embarked"],inplace=True)

# Filling Age with its mean value
titanic_data.fillna(titanic_data["Age"].mean(), inplace=True)

# Filling Cabin with 0
titanic_data.fillna(titanic_data["Cabin"].mode(),inplace=True)

In [ ]:
# Check Null Values again

titanic_data.isna().sum()

In [ ]:
# Let's check the skewness

titanic_data.skew()

As we discussed that SibSp & Parch are classifiation columns, So I have to go with Fare column for making skewness correct

In [ ]:
# Let's separate the numerica columns and categorical columns

numerical = titanic_data.drop(columns=["Sex", "Cabin", "Embarked"])
categorical = titanic_data[["Sex", "Cabin", "Embarked"]]

In [ ]:
# Let's visualizing the skewness with Distplot

plt.figure(figsize = (10,15), facecolor='white')
graphplot = 1
for column in numerical:
    if graphplot <=6:
        ax = plt.subplot(2,3,graphplot)
        sns.distplot(numerical[column])
        plt.xlabel(column, fontsize=15)
    graphplot+=1
plt.show()

Fare column has skewness and other column have classification problems

In [ ]:
# Let's check the outliers

plt.figure(figsize = (15,10), facecolor='white')
graphplot = 1
for column in numerical:
    if graphplot<=6:
        ax = plt.subplot(2,3,graphplot)
        sns.boxplot(numerical[column])
        plt.xlabel(column,fontsize=15)
    graphplot+=1
plt.show()

Outliers are present in Fare column.

In [ ]:
# Let's remove the outliers

from scipy.stats import zscore

z = np.abs(zscore(numerical))
titanic_data = titanic_data[(z<3).all(axis=1)]
titanic_data.shape

In [ ]:
# Let's check skewness again

titanic_data.skew()

Skewness is still pending in Fare column. Let's apply another method

In [ ]:
# Let's remove the pending skewness

for index in titanic_data.skew().index:
    if titanic_data.skew().loc[index]>0.5:
        titanic_data[index] = np.sqrt(titanic_data[index])
    if titanic_data.skew().loc[index]<-0.5:
        titanic_data[index] = np.cbrt(titanic_data[index])

In [ ]:
# Let's check skewness again..

titanic_data.skew()

Skewness is still pending in Fare column. Let's apply Power Transformer to remove it...

In [ ]:
# Let's apply Power Transformer to remove the skewness

from sklearn.preprocessing import PowerTransformer

features = ["Pclass","Age", "SibSp", "Parch","Fare"]

scaler = PowerTransformer(method='yeo-johnson')
titanic_data[features] = scaler.fit_transform(titanic_data[features].values)

In [ ]:
titanic_data.skew()

It seems that skewness has gone from Fare column. Now Let's deal with categorical column.

In [ ]:
# Let's separate the input and output

x = titanic_data.drop("Survived", axis=1)
y = titanic_data["Survived"]

In [ ]:
# Let's check the correlation between features and target

x.corrwith(y)

In [ ]:
# Correlation analysis by ploting a graph between features and target

x.corrwith(y).plot(kind='bar',grid=True, figsize=(8,5),rot=60,title="Correlation Between Features & Targer")

2 features is correlated in negative and 3 features in correlated in positive but not any features is highly correlated to the target. Now, let's check the multicollinearity between features

In [ ]:
# Multicollinearity between features

corr = titanic_data.corr()
corr

In [ ]:
# Let's plot a heatmap for better analysis the multicollinearity

plt.figure(figsize = (10,10))
sns.heatmap(corr,cbar=True,square=True,annot=True,cbar_kws={"shrink":.82},
            annot_kws={"size":10},cmap="Blues",fmt=".2f",linewidths=.2)
plt.xlabel("Multicollinearity between features",fontsize=20)
plt.show()

Now it is clear that no any single feature is correlated to another feature. Let's check VIF Score for more analysis.

In [ ]:
# Converting categorical column to numerical column using OneHotEncoder

cat_to_conv = ["Sex", "Cabin", "Embarked"]

titanic_data = pd.get_dummies(titanic_data, columns=cat_to_conv)

titanic_data.shape

In [ ]:
# Dataset after converting categorical columns to numerical columns

titanic_data.head()

In [ ]:
# Let's separate the input and output again..

X = titanic_data.drop(columns = ["Survived"],axis=1)
Y = titanic_data["Survived"]

In [ ]:
# Let's do Scaling first before proceeding to the VIF Score

from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
x_scaled = scalar.fit_transform(X)

x_scaled

In [ ]:
# Let's check the VIF Score first..

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Score"] = [variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif["Features"] = X.columns
                    
vif.head(5)

All the VIF Scores are less than 5. Now it is cross verified that multicollinearity is not exists between the features. Let's do the analysis of target column now.

In [ ]:
# Target Column (Survived) Analysis

titanic_data["Survived"].value_counts()

In [ ]:
# Let's plot countplot for better analysis

sns.countplot(titanic_data["Survived"])

It seems that class is not balance. Let's check the percentage of each using pie chart

In [ ]:
# let's plot a pie chart to check the contribution of each

titanic_data["Survived"].value_counts().plot.pie(figsize=(5,5), autopct="%1.2f",startangle=90,
                                                 labels=["Not Survived","Survived"],fontsize=20)

There is big difference between classes. Let's do the Oversampling to balance the classes

In [ ]:
# Oversampling for class Imbalance


from imblearn.over_sampling import SMOTE

SM = SMOTE()

X,Y = SM.fit_resample(X,Y)

Y.value_counts()

In [ ]:
# Let's check countplot again...

sns.countplot(Y)

Now It is clear that class are balanced.

In [ ]:
# Let's do scaling again for the latest oversampled data

from sklearn.preprocessing import StandardScaler

sca = StandardScaler()
x_scal = sca.fit_transform(X)

x_scal

In [ ]:
# Shape of final data

titanic_data.shape

In [ ]:
# There are so many columns. So, Let's use the PCA

from sklearn.decomposition import PCA
from sklearn.metrics import explained_variance_score

x_pca = PCA()
x_pca.fit(x_scal)

var_cumu = np.cumsum(x_pca.explained_variance_ratio_)*100

k = np.argmax(var_cumu>95)
print("Number of component explaining 95% variance : ",k)

plt.xlabel("Principle Component", fontsize=15)
plt.ylabel("Cumulative Explained Variance", fontsize=15)
plt.axvline(x=k, color='k', linestyle='--')
plt.axhline(y=95, color='r', linestyle='--')
plt.plot(var_cumu)
plt.plot()

In [ ]:
# Finalysing the features with PCA for model buildings

Pca = PCA(n_components = 131)
df_x = Pca.fit_transform(x_scal)

df_x

In [ ]:
# Let's import the necessary libraries for models

from sklearn.metrics import accuracy_score, plot_roc_curve, classification_report
from sklearn.model_selection import train_test_split, cross_val_score


In [ ]:
# Let's find the best random state

def model(mod):
    max_auc = 0
    max_state = 0
    for i in range(1,201):
        x_train,x_test,y_train,y_test = train_test_split(df_x,Y,test_size = 0.25, random_state =i)
        mod.fit(x_train,y_train)
        y_pred = mod.predict(x_test)
        auc_scor = accuracy_score(y_test, y_pred)
        if auc_scor>max_auc:
            max_auc=auc_scor
            max_state=i
    print("Best Accuracy Score corresponding to ",max_state,"is ",auc_scor)
    print("Cross Validation Score : ",cross_val_score(mod,df_x,Y,cv=5).mean())
    print("Classifiation Report-->\n\n",classification_report(y_test,y_pred))

In [ ]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
model(LR)

In [ ]:
# Models Format

def models(mods):
    x_train,x_test,y_train,y_test = train_test_split(df_x,Y,test_size = 0.25, random_state =41)
    mods.fit(x_train,y_train)
    y_preds = mods.predict(x_test)
    auc_score = accuracy_score(y_test, y_preds)
    print("Best Accuracy Score corresponding to ",auc_score)
    print("Cross Validation Score : ",cross_val_score(mods,df_x,Y,cv=5).mean())
    print("Classifiation Report-->\n\n",classification_report(y_test,y_preds))          

In [ ]:
# Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier

DTR = DecisionTreeClassifier()

models(DTR)

In [ ]:
# K-Neighbors Classifier

from sklearn.neighbors import KNeighborsClassifier

KNC = KNeighborsClassifier()

models(KNC)

In [ ]:
# Gradient Boosting Classifier

from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()

models(GBC)

In [ ]:
# Bagging Classifier

from sklearn.ensemble import BaggingClassifier

BC = BaggingClassifier()

models(BC)

In [ ]:
# Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()

models(RFC)

****Let's check ROC AUC Curve for the fitted model****

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df_x,Y,test_size = 0.25, random_state =41)

In [ ]:
disp = plot_roc_curve(LR, x_test, y_test)
plot_roc_curve(DTR, x_test, y_test, ax=disp.ax_)
plot_roc_curve(KNC, x_test, y_test, ax=disp.ax_)
plot_roc_curve(BC, x_test, y_test, ax=disp.ax_)
plot_roc_curve(RFC, x_test, y_test, ax=disp.ax_)
plot_roc_curve(GBC, x_test, y_test, ax=disp.ax_)

plt.legend(prop={"size":10},loc="lower right")
plt.show()

**After analyzing ROC AUC Curve the Accuray Scores & Cross Validatio Scores of above 6 models, I choose Logistic Regression becasue this is the only model which is not getting the difference in accuracy score after cross validation and It is looking stable with it's score. Now let's try to increase the score by applying hyperparameter tuning**

In [ ]:
# Hyperparameter Tuning with Logistic Regression

from sklearn.model_selection import GridSearchCV

param_grid = {"penalty" : ['l1','l2'], "intercept_scaling" : [1,2,3],
             "random_state" : [81,131,151], "max_iter" : [100,200]}

grid_srch = GridSearchCV(LR, param_grid=param_grid)
grid_srch.fit(df_x,Y)
grid_srch.best_params_

**Final Model**

In [ ]:
Final_Model = LogisticRegression(intercept_scaling=1, max_iter=100, penalty='l2', random_state=81)

x_train,x_test,y_train,y_test = train_test_split(df_x,Y,test_size = 0.25, random_state =41)
Final_Model.fit(x_train,y_train)
y_preds = Final_Model.predict(x_test)
auc_scr = accuracy_score(y_test, y_preds)
print("Best Accuracy Score corresponding to ",auc_scr)
print("Classifiation Report-->\n\n",classification_report(y_test,y_preds))

**Saving The Model**

In [ ]:
import joblib

joblib.dump(Final_Model,"Final_Titanic_Project_Model.plk")